# Loading large datasets

**Learning Objectives**
  - Understand difference between loading data entirely in-memory and loading in batches from disk
  - Practice loading a `.csv` file from disk in batches using the `tf.data` module
 
## Introduction

In the previous notebook, we read the the whole taxifare .csv files into memory, specifically a Pandas dataframe, before invoking `tf.data.from_tensor_slices` from the tf.data API. We could get away with this because it was a small sample of the dataset, but on the full taxifare dataset this wouldn't be feasible.

In this notebook we demonstrate how to read .csv files directly from disk, one batch at a time, using `tf.data.TextLineDataset`

Run the following cell and restart the kernel if needed

In [1]:
# Ensure that we have Tensorflow 1.13.1 installed.
!pip3 freeze | grep tensorflow==1.13.1 || pip3 install tensorflow==1.13.1

    100% |████████████████████████████████| 92.5MB 199kB/s eta 0:00:01  1% |▌                               | 1.5MB 42.3MB/s eta 0:00:03    4% |█▌                              | 4.4MB 29.2MB/s eta 0:00:04    6% |██▏                             | 6.1MB 37.8MB/s eta 0:00:03    15% |█████                           | 14.6MB 40.8MB/s eta 0:00:02    28% |█████████▏                      | 26.7MB 43.1MB/s eta 0:00:02    40% |█████████████                   | 37.6MB 10.1MB/s eta 0:00:06    41% |█████████████▍                  | 38.7MB 39.8MB/s eta 0:00:02    43% |██████████████                  | 40.7MB 12.2MB/s eta 0:00:05    53% |█████████████████               | 49.1MB 37.8MB/s eta 0:00:02    60% |███████████████████▎            | 55.8MB 37.9MB/s eta 0:00:01    64% |████████████████████▌           | 59.3MB 39.2MB/s eta 0:00:01    68% |██████████████████████          | 63.6MB 41.3MB/s eta 0:00:01    73% |███████████████████████▋        | 68.2MB 38.4MB/s eta 0:00:01    77% |███████████████████

In [2]:
import tensorflow as tf
import shutil
print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.13.1


In [4]:
tf.enable_eager_execution()

## Input function reading from CSV

We define `read_dataset()` which given a csv file path returns a `tf.data.Dataset` in which each row represents a (features,label) in the Estimator API required format 
- features: A python dictionary. Each key is a feature column name and its value is the tensor containing the data for that feature
- label: A Tensor containing the labels

We then invoke `read_dataset()` function from within the `train_input_fn()` and `eval_input_fn()`. The remaining code is as before.

#### **Exercise 1**

In the next cell, implement a `parse_row` function that takes as input a csv row (as a string) 
and returns a tuple (features, labels) as described above.

First, use the [tf.decode_csv function](https://www.tensorflow.org/api_docs/python/tf/io/decode_csv) to read in the features from a csv file. Next, once `fields` has been read from the `.csv` file, create a dictionary of features and values. Lastly, define the label and remove it from the `features` dict you created. This can be done in one step with pythons pop operation.

The column names and the default values you'll need for these operations are given by global variables `CSV_COLUMN_NAMES`
and `CSV_DEFAULTS`. The labels are stored in the first column.

In [7]:
CSV_COLUMN_NAMES = ["fare_amount","dayofweek","hourofday","pickuplon","pickuplat","dropofflon","dropofflat"]
CSV_DEFAULTS = [[0.0],[1],[0],[-74.0], [40.0], [-74.0], [40.7]]

def parse_row(row):
    fields = tf.decode_csv(row, CSV_DEFAULTS)# TODO: Your code goes here
    features = dict(zip(CSV_COLUMN_NAMES, fields))# TODO: Your code goes here
    labels = features.pop("fare_amount") # TODO: Your code goes here
    return features, labels

Run the following test to make sure your implementation is correct

In [8]:
a_row = "0.0,1,0,-74.0,40.0,-74.0,40.7"
features, labels = parse_row(a_row)

assert labels.numpy() == 0.0
assert features["pickuplon"].numpy() == -74.0
print("You rock!")

You rock!


#### **Exercise 2**

Use the function `parse_row` you implemented in the previous exercise to 
implement a `read_dataset` function that
- takes as input the path to a csv file
- returns a `tf.data.Dataset` object containing the features, labels

Assume that the .csv file has a header, and that your `read_dataset` will skip it. Have a look at the [tf.data.TextLineDataset documentation](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) to see what variables to pass when initializing the dataset pipeline. Then use the `parse_row` operation we created above to read the values from the .csv file

In [9]:
def read_dataset(csv_path):  
    dataset = tf.data.TextLineDataset(csv_path).skip(1) # TODO: Your code goes here
    dataset = dataset.map(parse_row) # TODO: Your code goes here
    return dataset

### Tests

Let's create a test dataset to test our function.

In [10]:
%%writefile test.csv
fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat
28,1,0,-73.0,41.0,-74.0,20.7
12.3,1,0,-72.0,44.0,-75.0,40.6
10,1,0,-71.0,41.0,-71.0,42.9

Writing test.csv


You should be able to iterate over what's returned by `read_dataset`. We'll print the `dropofflat` and `fare_amount` for each entry in `./test.csv`

In [11]:
for feature, label in read_dataset("./test.csv"):
    print("dropofflat:", feature["dropofflat"].numpy())
    print("fare_amount:", label.numpy())

Instructions for updating:
Colocations handled automatically by placer.
dropofflat: 20.7
fare_amount: 28.0
dropofflat: 40.6
fare_amount: 12.3
dropofflat: 42.9
fare_amount: 10.0


Run the following test cell to make sure you function works properly:

In [12]:
dataset= read_dataset("./test.csv")
dataset_iterator = dataset.make_one_shot_iterator()
features, labels = dataset_iterator.get_next()

assert features['dayofweek'].numpy() == 1
assert labels.numpy() == 28
print("You rock!")

You rock!


#### **Exercise 3**

In the code cell below, implement a `train_input_fn` function that
- takes as input a path to a csv file along with a batch_size
- returns a dataset object that shuffle the rows and returns them in batches of `batch_size`

**Hint:** Reuse the `read_dataset` function you implemented above. 

Once you've initialized the `dataset`, be sure to add a step to `shuffle`, `repeat` and `batch` to your pipeline.

In [23]:
def train_input_fn(csv_path, batch_size = 128):
    dataset = read_dataset(csv_path)# TODO: Your code goes here
    dataset = dataset.shuffle(buffer_size = 1000).batch (batch_size = batch_size).repeat(count = None)# TODO: Your code goes here
    return dataset

#### **Exercise 4**

Next, implement as `eval_input_fn` similar to the `train_input_fn` you implemented above. Remember, the only difference is that this function does not need to shuffle the rows.

In [16]:
def eval_input_fn(csv_path, batch_size = 128):
    dataset = read_dataset(csv_path) # TODO: Your code goes here
    dataset = dataset.batch (batch_size = batch_size).repeat(count = 1)# TODO: Your code goes here
    return dataset

## Create feature columns

The features of our models are the following:

In [17]:
FEATURE_NAMES = CSV_COLUMN_NAMES[1:] # all but first column
print(FEATURE_NAMES)

['dayofweek', 'hourofday', 'pickuplon', 'pickuplat', 'dropofflon', 'dropofflat']


#### **Exercise 5**

In the cell below, create a variable called `feature_cols` which contains a list of the appropriate `tf.feature_column` to be passed to a `tf.estimator`.

In [18]:
feature_cols = [ tf.feature_column.numeric_column(i) for i in FEATURE_NAMES] # TODO: Your code goes here
print(feature_cols)

[NumericColumn(key='dayofweek', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='hourofday', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='pickuplon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='pickuplat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='dropofflon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='dropofflat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


### Choose Estimator 

#### **Exercise 6**

In the cell below, create an instance of a `tf.estimator.DNNRegressor` such that
- it has two layers of 10 units each
- it uses the features defined in the previous exercise
- it saves the trained model into the directory `./taxi_trained`
- it has a random seed set to 1 for replicability and debugging

Have a look at [the documentation for Tensorflow's DNNRegressor](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNRegressor) to remind you of the implementation.

**Hint:** Remember, the random seed is set by passing a `tf.estimator.RunConfig` object
  to the `config` parameter of the `tf.estimator`.

In [19]:
OUTDIR = "taxi_trained"

model = tf.estimator.DNNRegressor(hidden_units = [10, 10], 
                                  feature_columns = feature_cols, 
                                  model_dir = OUTDIR,
                                  config = tf.estimator.RunConfig(tf_random_seed = 1))  # TODO: Your code goes here

INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f12346ab160>, '_log_step_count_steps': 100, '_eval_distribute': None, '_task_type': 'worker', '_global_id_in_cluster': 0, '_tf_random_seed': 1, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_experimental_distribute': None, '_model_dir': 'taxi_trained', '_service': None, '_task_id': 0, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_evaluation_master': '', '_master': '', '_train_distribute': None, '_is_chief': True, '_save_checkpoints_steps': None, '_device_fn': None, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
}


## Train

Next, we'll train the model.

#### **Exercise 7**

Complete the code in the cell below to train the `DNNRegressor` model you instantiated above on our data. Have a look at [the documentation for the `train` method of the `DNNRegressor`](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNRegressor#train) to see what variables you should pass. You'll use the `train_input_function` you created above and the `./taxi-train.csv` dataset. 

If you train your model for 500 steps. How many epochs of the dataset does this represent? 

In [24]:
%%time
tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
shutil.rmtree(path = OUTDIR, ignore_errors = True) # start fresh each time

model.train(
    input_fn = lambda: train_input_fn(csv_path = "./taxi-train.csv"), # TODO: Your code goes here,
    steps = 500 # TODO: Your code goes here
)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into taxi_trained/model.ckpt.


NotFoundError: ./taxi-train.csv; No such file or directory
	 [[node IteratorGetNext (defined at /usr/local/envs/py3env/lib/python3.5/site-packages/tensorflow_estimator/python/estimator/util.py:110) ]]

Caused by op 'IteratorGetNext', defined at:
  File "/usr/local/envs/py3env/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/envs/py3env/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/envs/py3env/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/envs/py3env/lib/python3.5/site-packages/tornado/ioloop.py", line 888, in start
    handler_func(fd_obj, events)
  F

## Evaluate

Lastly, we'll evaluate our model.

#### **Exercise 8**

In the cell below, evaluate the model using its `.evaluate` method and the `eval_input_fn` function you implemented above on the `/.taxi-valid.csv` dataset. Capture the result of running evaluation on the evaluation set in a variable called `metrics`. Then, extract the `average_loss` for the dictionary returned by `model.evaluate` and contained in `metrics`. This is the RMSE.

In [ ]:
metrics = # TODO: Your code goes here
print("RMSE on dataset = {}".format(# TODO: Your code goes here))

## Challenge exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Unlike in the challenge exercise for b_estimator.ipynb, assume that your measurements of r, h and V are all rounded off to the nearest 0.1. Simulate the necessary training dataset. This time, you will need a lot more data to get a good predictor.

Hint (highlight to see):
<p style='color:white'>
Create random values for r and h and compute V. Then, round off r, h and V (i.e., the volume is computed from the true value of r and h; it's only your measurement that is rounded off). Your dataset will consist of the round values of r, h and V. Do this for both the training and evaluation datasets.
</p>

Now modify the "noise" so that instead of just rounding off the value, there is up to a 10% error (uniformly distributed) in the measurement followed by rounding off.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License